# Probelm 1

In [39]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up. In the first exercise we want you to audit
the datatypes that can be found in some particular fields in the dataset.
The possible types of values can be:
- NoneType if the value is a string "NULL" or an empty string ""
- list, if the value starts with "{"
- int, if the value can be cast to int
- float, if the value can be cast to float, but CANNOT be cast to int.
   For example, '3.23e+07' should be considered a float because it can be cast
   as float but int('3.23e+07') will throw a ValueError
- 'str', for all other values

The audit_file function should return a dictionary containing fieldnames and a 
SET of the types that can be found in the field. e.g.
{"field1": set([type(float()), type(int()), type(str())]),
 "field2": set([type(str())]),
  ....
}
The type() function returns a type object describing the argument given to the 
function. You can also use examples of objects to create type objects, e.g.
type(1.1) for a float: see the test function below for examples.

Note that the first three rows (after the header row) in the cities.csv file
are not actual data points. The contents of these rows should note be included
when processing data types. Be sure to include functionality in your code to
skip over or detect these rows.
"""
import codecs
import csv
import json
import pprint

CITIES = 'cities.csv'

FIELDS = ["name", "timeZone_label", "utcOffset", "homepage", "governmentType_label",
          "isPartOf_label", "areaCode", "populationTotal", "elevation",
          "maximumElevation", "minimumElevation", "populationDensity",
          "wgs84_pos#lat", "wgs84_pos#long", "areaLand", "areaMetro", "areaUrban"]

def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        pass
    
    try:
        import unicodedata
        unicodedata.numeric(string)
        return True
    except (TypeError, ValueError):
        pass
    
    return False

def audit_file(filename, fields):
    fieldtypes = {}

    # YOUR CODE HERE
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        for i in range(3):
            reader.next()
                        
        first_info = reader.next()
        
        # initialise dictionary as a set for a field
        for field in fields:
            fieldtypes[field] = set()
        
        for row in reader:
            for field in fields:
                if row[field] == 'NULL' or row[field] == '':
                    fieldtypes[field].add(type(None))
                    #print('{0:5} -- NULL: {1}'.format('', fieldtypes[field]))

                elif is_number(row[field]):
                    #print('{0:5} -> digit: {1}'.format('', first_info[field]))

                    if row[field].isdigit():
                        fieldtypes[field].add(type(int()))
                        #print('{0:5} -- int: {1}'.format('', fieldtypes[field]))
                    else:
                        fieldtypes[field].add(type(float()))
                        #print('{0:5} -- float: {1}'.format('', fieldtypes[field]))

                elif row[field].startswith('{'):
                    fieldtypes[field].add(type(list()))
                    #print('{0:5} -- list: {1}'.format('', fieldtypes[field]))

                else:
                    fieldtypes[field].add(type(str()))

    return fieldtypes


def test():
    fieldtypes = audit_file(CITIES, FIELDS)

    pprint.pprint(fieldtypes)

    assert fieldtypes["areaLand"] == set([type(1.1), type([]), type(None)])
    assert fieldtypes['areaMetro'] == set([type(1.1), type([]), type(None)])
    print('Great Job')
    
if __name__ == "__main__":
    test()

{'areaCode': set([<type 'float'>,
                  <type 'int'>,
                  <type 'list'>,
                  <type 'NoneType'>,
                  <type 'str'>]),
 'areaLand': set([<type 'float'>, <type 'list'>, <type 'NoneType'>]),
 'areaMetro': set([<type 'float'>, <type 'list'>, <type 'NoneType'>]),
 'areaUrban': set([<type 'float'>, <type 'list'>, <type 'NoneType'>]),
 'elevation': set([<type 'float'>, <type 'list'>, <type 'NoneType'>]),
 'governmentType_label': set([<type 'list'>, <type 'NoneType'>, <type 'str'>]),
 'homepage': set([<type 'list'>, <type 'NoneType'>, <type 'str'>]),
 'isPartOf_label': set([<type 'list'>, <type 'NoneType'>, <type 'str'>]),
 'maximumElevation': set([<type 'float'>, <type 'list'>, <type 'NoneType'>]),
 'minimumElevation': set([<type 'float'>, <type 'NoneType'>]),
 'name': set([<type 'list'>, <type 'NoneType'>, <type 'str'>]),
 'populationDensity': set([<type 'float'>, <type 'list'>, <type 'NoneType'>]),
 'populationTotal': set([<type 'int'>, <t

---
# Problem 2

In [15]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

Since in the previous quiz you made a decision on which value to keep for the
"areaLand" field, you now know what has to be done.

Finish the function fix_area(). It will receive a string as an input, and it
has to return a float representing the value of the area or None.
You have to change the function fix_area. You can use extra functions if you
like, but changes to process_file will not be taken into account.
The rest of the code is just an example on how this function can be used.
"""
import codecs
import csv
import json
import pprint

CITIES = 'cities.csv'

def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        pass
    
    try:
        import unicodedata
        unicodedata.numeric(string)
        return True
    except (TypeError, ValueError):
        pass
    
    return False

def fix_area(area):
    # YOUR CODE HERE
    if is_number(area):
        area = float(area)
        
    elif area == 'NULL':
        area = None

    elif area.startswith('{'):
        areas = area[1:-1].split('|')
        #areas = map(lambda x: float(x), areas)
        #area = sum(areas) / float(len(areas))
        
        for idx, value in enumerate(areas):
            if idx == 0:
                significant_v = value
                significant_l = len(value)

            if significant_l < len(value):
                significant_l = len(value)
                significant_v = value
                
            area = float(value)

    return area

def process_file(filename):
    # CHANGES TO THIS FUNCTION WILL BE IGNORED WHEN YOU SUBMIT THE EXERCISE
    data = []

    with open(filename, "r") as f:
        reader = csv.DictReader(f)

        #skipping the extra metadata
        for i in range(3):
            l = reader.next()

        # processing file
        for line in reader:
            # calling your function to fix the area value
            if "areaLand" in line:
                line["areaLand"] = fix_area(line["areaLand"])

            data.append(line)

    #print('len:', len(data))
    return data


def test():
    data = process_file(CITIES)

    print "Printing three example results:"
    for n in range(5,8):
        pprint.pprint(data[n]["areaLand"])

    assert data[3]["areaLand"] == None
    assert data[8]["areaLand"] == 55166700.0
    assert data[20]["areaLand"] == 14581600.0
    assert data[33]["areaLand"] == 20564500.0    


if __name__ == "__main__":
    test()

Printing three example results:
None
None
None


AssertionError: 

---
# Problem 3

In [16]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

In the previous quiz you recognized that the "name" value can be an array (or
list in Python terms). It would make it easier to process and query the data
later if all values for the name are in a Python list, instead of being
just a string separated with special characters, like now.

Finish the function fix_name(). It will recieve a string as an input, and it
will return a list of all the names. If there is only one name, the list will
have only one item in it; if the name is "NULL", the list should be empty.
The rest of the code is just an example on how this function can be used.
"""
import codecs
import csv
import pprint

CITIES = 'cities.csv'

def fix_name(name):

    # YOUR CODE HERE
    if name == 'NULL':
        name = []
    elif name.startswith('{'):
        names_list = []
        names_str = name[1:-1].split('|')
        
        for name_str in names_str:
            names_list.append(name_str)
            
        name = names_list
    else:
        name = [name]
        
    return name


def process_file(filename):
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        #skipping the extra metadata
        for i in range(3):
            l = reader.next()
        # processing file
        for line in reader:
            # calling your function to fix the area value
            if "name" in line:
                line["name"] = fix_name(line["name"])
            data.append(line)
    return data


def test():
    data = process_file(CITIES)

    print "Printing 20 results:"
    for n in range(20):
        pprint.pprint(data[n]["name"])

    assert data[14]["name"] == ['Negtemiut', 'Nightmute']
    assert data[9]["name"] == ['Pell City Alabama']
    assert data[3]["name"] == ['Kumhari']

if __name__ == "__main__":
    test()

Printing 20 results:
['Kud']
['Kuju']
['Kumbhraj']
['Kumhari']
['Kundagola',
 'Kundgol \xe0\xb2\x95\xe0\xb3\x81\xe0\xb2\x82\xe0\xb2\xa6\xe0\xb2\x97\xe0\xb3\x8b\xe0\xb2\xb3']
['Kunigal']
['Kunzer']
['Kurduvadi',
 '\xe0\xa4\x95\xe0\xa5\x81\xe0\xa4\xb0\xe0\xa5\x8d\xe0\xa4\xa1\xe0\xa5\x81\xe0\xa4\xb5\xe0\xa4\xbe\xe0\xa4\xa1\xe0\xa5\x80']
['Kurgunta']
['Kurinjipadi']
['Kurud']
['Kushtagi']
['Ladnun',
 '\xe0\xa4\xb2\xe0\xa4\xbe\xe0\xa4\xa1\xe0\xa4\xa8\xe0\xa5\x82\xe0\xa4\x81']
['Lahar', '\xe0\xa4\xb2\xe0\xa4\xb9\xe0\xa4\xbe\xe0\xa4\xb0']
['Laharpur']
['Lakheri']
['Lakhipur']
['Laksar', '\xe0\xa4\xb2\xe0\xa4\x95\xe0\xa5\x8d\xe0\xa4\xb8\xe0\xa4\xb0']
['Lalkuan']
['Lalsot']


AssertionError: 

---
# Problem 4

In [17]:
aaa = 'aser#1234'

aaa.replace('aser#', '')

'1234'